In [138]:
from IPython.display import display, display_html
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [164]:
!!pip install urllib3

['Requirement already satisfied: urllib3 in ./.venv/lib/python3.8/site-packages (1.26.3)',
 'Requirement already satisfied: json2table in ./.venv/lib/python3.8/site-packages (1.1.5)',
 'WARNING: You are using pip version 20.2.3; however, version 21.0.1 is available.',
 "You should consider upgrading via the '/private/tmp/IPrecon/.venv/bin/python3 -m pip install --upgrade pip' command."]

In [165]:
SHODAN_APIKEY = ""
SCAMALYTICS_APIKEY = ""

In [166]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [222]:
from IPython.core.display import display, display_html, HTML
from IPython.display import Javascript, display
import ipywidgets as widgets

IPwidget=widgets.Text(
    value='',
    placeholder='<IP Address>',
    description='IP Address:',
    disabled=False
)
display(IPwidget)

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

button = widgets.Button(description="Run")
button.on_click(run_all)
display(button)

Text(value='', description='IP Address:', placeholder='<IP Address>')

Button(description='Run', style=ButtonStyle())

In [221]:
from urllib.request import urlopen
from urllib.request import Request
import urllib3
import json
import socket


http = urllib3.PoolManager()

INPUT_IP=IPwidget.value

try:
  socket.inet_aton(INPUT_IP)
    # legal
  url = f"https://api12.scamalytics.com/netscylla?ip={INPUT_IP}&key={SCAMALYTICS_APIKEY}"
  r = http.request(
      'GET',
      url
  )
  data=json.loads(r.data.decode('utf-8'))
  if(data['risk']=="low"):
      row_col="green"
  if(data['risk']=="medium"):
      row_col="orange"
  if(data['risk']=="high"):   
      row_col="red"
  display(HTML("<h1>Risk Score</h1>"))
  display(HTML("<table>"))
  display(HTML("<tr bgcolor="+row_col+"><td><p style=\"color:white\">Risk</p></td><td><p style=\"color:white\">"+str(data['risk'])+"</p></td><td><p style=\"color:white\">Fraud Score</p></td><td><p style=\"color:white\">"+str(data['score'])+"</p></td></tr>"))
  display(HTML("</table>"))

  url = f"https://api.shodan.io/shodan/host/{INPUT_IP}?key={SHODAN_APIKEY}"
  r = http.request(
      'GET',
      url
  )

  display(HTML("<h1>Shodan</h1>"))

  data=json.loads(r.data.decode('utf-8'))
  if (len(data) < 25):
    display(HTML("<p>No Data Found!</p>"))
  else:
    display(HTML("<table>"))
    display(HTML("<tr><td>ASN</td><td>"+str(data['asn'])+"</td></tr>"))
    display(HTML("<tr><td>IP</td><td>"+str(data['ip_str'])+"</td></tr>"))
    display(HTML("<tr><td>ISP</td><td>"+str(data['isp'])+"</td></tr>"))
    display(HTML("<tr><td>City</td><td>"+data['city']+", "+data['region_code']+", "+data['country_code']+"</td></tr>"))
    display(HTML("<tr><td>Long+Lat</td><td>"+str(data['longitude'])+", "+str(data['latitude'])+"</td></tr>"))
    display(HTML("<tr><td>IP</td><td>"+str(data['country_name'])+"</td></tr>"))
    display(HTML("<tr><td>Hostnames</td><td>"+str(data['hostnames'])+"</td></tr>"))
    display(HTML("<tr><td>TCP Ports</td><td>"+str(data['ports'])+"</td></tr>"))
    for i in range(len(data['ports'])):
      display(HTML("<tr><td>"+str(data['data'][i]['port'])+"</td><td>"+data['data'][i]['product']+"/"+str(data['data'][i]['version'])+"</td></tr>"))
    display(HTML("</table>"))
    
  display(HTML("<h1>URLHaus</h1>"))  
  #https://urlhaus-api.abuse.ch/v1/host/
  url = "https://urlhaus-api.abuse.ch/v1/host/"
  r = http.request(
      'POST',
      url,
      fields={'host': INPUT_IP} 
  )
  data=json.loads(r.data.decode('utf-8'))
  
  display(HTML("<table>"))
  display(HTML("<tr><td>Firstseen</td><td>"+str(data['firstseen'])+"</td></tr>"))
  for i in range(len(data['urls'])):  
    display(HTML("<tr><td>Urlhaus_reference</td><td>"+str(data['urls'][i]['urlhaus_reference'])+"</td></tr>"))
    display(HTML("<tr><td>URL</td><td>"+str(data['urls'][i]['url'])+"</td></tr>"))
    display(HTML("<tr><td>Status</td><td>"+str(data['urls'][i]['url_status'])+"</td></tr>")) 
    display(HTML("<tr><td>Threat</td><td>"+str(data['urls'][i]['threat'])+"</td></tr>"))
    display(HTML("<tr><td>Tags</td><td>"+str(data['urls'][i]['tags'])+"</td></tr>"))  
  display(HTML("</table>"))
except socket.error:
  display(HTML("Not a valid IP"))


Risk,low,Fraud Score,0


Firstseen,2021-02-17 13:51:03 UTC


Urlhaus_reference,https://urlhaus.abuse.ch/url/1015887/


URL,http://177.86.234.13:55661/bin.sh


Status,offline


Threat,malware_download


Tags,"['32-bit', 'arm', 'elf']"
